task name: NormalizeMinMax
task description: Implement the NormalizeMinMax example using ML.NET

In [ ]:
#r "nuget:Microsoft.ML"

In [ ]:
using System;
using System.Collections.Generic;
using System.Collections.Immutable;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using static Microsoft.ML.Transforms.NormalizingTransformer;


In [ ]:
var mlContext = new MLContext();

In [ ]:
public class DataPoint
{
    [VectorType(4)]
    public float[] Features { get; set; }
}

var samples = new List<DataPoint>()
{
    new DataPoint(){ Features = new float[4] { 1, 1, 3, 0} },
    new DataPoint(){ Features = new float[4] { 2, 2, 2, 0} },
    new DataPoint(){ Features = new float[4] { 0, 0, 1, 0} },
    new DataPoint(){ Features = new float[4] {-1,-1,-1, 1} }
};
var data = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
var normalize = mlContext.Transforms.NormalizeMinMax("Features", fixZero: false);
var normalizeFixZero = mlContext.Transforms.NormalizeMinMax("Features", fixZero: true);

In [ ]:
var normalizeTransform = normalize.Fit(data);
var transformedData = normalizeTransform.Transform(data);
var normalizeFixZeroTransform = normalizeFixZero.Fit(data);
var fixZeroData = normalizeFixZeroTransform.Transform(data);

In [ ]:
var column = transformedData.GetColumn<float[]>("Features").ToArray();
foreach (var row in column)
    Console.WriteLine(string.Join(", ", row.Select(x => x.ToString("f4"))));

var columnFixZero = fixZeroData.GetColumn<float[]>("Features").ToArray();
foreach (var row in columnFixZero)
    Console.WriteLine(string.Join(", ", row.Select(x => x.ToString("f4"))));

In [ ]:
var transformParams = normalizeTransform.GetNormalizerModelParameters(0) as AffineNormalizerModelParameters<ImmutableArray<float>>;
Console.WriteLine($"The 1-index value in resulting array would be produced by:");
Console.WriteLine(" y = (x - (" + (transformParams.Offset.Length == 0 ? 0 : transformParams.Offset[1]) + ")) * " + transformParams.Scale[1]);